In [99]:
# --- ALWAYS RUN FIRST ---
from pathlib import Path
import os, time
import pandas as pd
import numpy as np

def set_project_root():
    p = Path.cwd()
    if p.name == "notebooks":
        os.chdir("..")
    print("Root:", Path.cwd())

set_project_root()

# Rutas estándar
BASE = Path.cwd()
RAW  = BASE / "data" / "raw"
PROC = BASE / "data" / "processed"
(REVIEW := PROC / "review").mkdir(parents=True, exist_ok=True)

# Archivos origen
T_TOP  = RAW / "top_twitch" / "Twitch_game_data.csv"
E_HIST = RAW / "esports_earnings" / "HistoricalEsportData.csv"
E_GEN  = RAW / "esports_earnings" / "GeneralEsportData.csv"
ALIAS_CSV = RAW / "esports_earnings" / "games_alias.csv"  # opcional: alias externo

for p in [T_TOP, E_HIST, E_GEN]:
    assert p.exists(), f"Falta archivo: {p}"
print("✅ Archivos origen localizados")

# Ventana temporal (YEAR_MAX se ajusta después al leer Twitch)
YEAR_MIN, YEAR_MAX = 2016, 2025

# Columnas del dataset anual final
COLS_YEARLY = [
    "JuegoNorm","Año","Género",
    "MediaViewers_K","PicoViewers_K","HorasVistas_M",
    "PremiosUSD_M","Torneos",
    "PremiosTotalesUSD_M","JugadoresTotales","TorneosTotales"
]

def tictoc(fn):
    def wrap(*a, **k):
        t0=time.time(); r=fn(*a, **k); dt=time.time()-t0
        print(f"⏱ {fn.__name__}: {dt:.2f}s")
        return r
    return wrap


Root: c:\Users\cubea\OneDrive\Escritorio\project_esports_occidente
✅ Archivos origen localizados


In [100]:
# Lector CSV robusto con separadores y encodings comunes + fallback
@tictoc
def read_csv_fast(path, usecols=None, dtypes=None):
    """
    Orden de intento:
      1) PyArrow (rápido, coma)
      2) Combinaciones encoding x separador [",", ";", "\\t", "|"]
      3) Sniff automático (engine='python')
      4) Excel (si el archivo no es CSV real)
    """
    # 1) PyArrow (solo coma)
    try:
        return pd.read_csv(path, usecols=usecols, dtype=dtypes, engine="pyarrow")
    except Exception:
        pass

    # 2) Combos encoding x separador
    encs = [None, "utf-8-sig", "cp1252", "latin1", "utf-16"]
    seps = [",", ";", "\t", "|"]
    for enc in encs:
        for sep in seps:
            try:
                return pd.read_csv(path, usecols=usecols, dtype=dtypes, encoding=enc, sep=sep)
            except Exception:
                continue

    # 3) Sniff automático
    for enc in encs:
        try:
            return pd.read_csv(path, usecols=usecols, encoding=enc, sep=None, engine="python")
        except Exception:
            continue

    # 4) Fallback Excel (por si el archivo es Excel o CSV mal guardado)
    try:
        return pd.read_excel(path, usecols=usecols, dtype=dtypes)
    except Exception:
        raise RuntimeError(f"No pude leer {path}. Revisa separador/encoding o si es Excel mal guardado.")

# Normalización de texto estable (claves)
def norm_text(s: pd.Series) -> pd.Series:
    return (s.astype(str)
              .str.normalize("NFKC")
              .str.strip()
              .str.replace(r"\s+", " ", regex=True)
              .str.casefold())

def to_int64(s):   return pd.to_numeric(s, errors="coerce").astype("Int64")
def to_float(s):   return pd.to_numeric(s, errors="coerce")

def downcast_num(df):
    for c in df.select_dtypes(include=["float64"]).columns:
        df[c] = pd.to_numeric(df[c], downcast="float")
    for c in df.select_dtypes(include=["int64"]).columns:
        df[c] = pd.to_numeric(df[c], downcast="integer")
    return df

def first_non_null(s: pd.Series):
    s = s.dropna()
    return s.iloc[0] if len(s) else np.nan

# Alias inicial (puedes ampliarlo) + fusión con CSV externo si existe
ALIAS = {
    "csgo":"counter-strike", "cs:go":"counter-strike", "counter strike":"counter-strike",
    "counter-strike 2":"counter-strike 2",
    "dota":"dota 2", "dota2":"dota 2",
    "lol":"league of legends", "league":"league of legends",
}
if ALIAS_CSV.exists():
    tmp = pd.read_csv(ALIAS_CSV)  # columnas: alias, target
    tmp["alias"]  = norm_text(tmp["alias"])
    tmp["target"] = norm_text(tmp["target"])
    ALIAS.update(dict(zip(tmp["alias"], tmp["target"])))

def apply_alias(s: pd.Series) -> pd.Series:
    s = norm_text(s)
    return s.replace(ALIAS)


In [101]:
# --- Twitch (juego/mes/año) ---
t_usecols = ["Game","Year","Month","Avg_viewers","Peak_viewers","Hours_watched"]
t_dtypes  = {"Game":"string","Year":"Int64","Month":"Int64",
             "Avg_viewers":"float64","Peak_viewers":"float64","Hours_watched":"float64"}
t_raw = read_csv_fast(T_TOP, usecols=t_usecols, dtypes=t_dtypes)

t = (t_raw.rename(columns={
        "Game":"Juego", "Year":"Año", "Month":"Mes",
        "Avg_viewers":"MediaViewers", "Peak_viewers":"PicoViewers",
        "Hours_watched":"HorasVistas"
     })[["Juego","Año","Mes","MediaViewers","PicoViewers","HorasVistas"]]
     .assign(
         JuegoNorm=lambda d: apply_alias(d["Juego"]),
         Año=lambda d: to_int64(d["Año"]),
         Mes=lambda d: to_int64(d["Mes"]).where(lambda s: s.between(1,12)),
         MediaViewers=lambda d: to_float(d["MediaViewers"]),
         PicoViewers=lambda d: to_float(d["PicoViewers"]),
         HorasVistas=lambda d: to_float(d["HorasVistas"])
     )
     .dropna(subset=["Mes"])
)

# Ajusta YEAR_MAX dinámicamente si Twitch llega más allá
if t["Año"].notna().any():
    YEAR_MAX = max(int(t["Año"].max()), YEAR_MAX)
print(f"🗓 Twitch años: {int(t['Año'].min())} → {YEAR_MAX}")

# --- Esports históricos (premios por juego/fecha -> año) ---
e_usecols = ["Game","Earnings","Tournaments","Date"]
e_dtypes  = {"Game":"string","Earnings":"float64","Tournaments":"float64","Date":"string"}
e_raw = read_csv_fast(E_HIST, usecols=e_usecols, dtypes=e_dtypes)

e = (e_raw.rename(columns={
        "Game":"Juego","Earnings":"PremiosUSD","Tournaments":"Torneos","Date":"Fecha"
     })[["Juego","PremiosUSD","Torneos","Fecha"]]
     .assign(
         Año=lambda d: pd.to_datetime(d["Fecha"], errors="coerce").dt.year.astype("Int64"),
         JuegoNorm=lambda d: apply_alias(d["Juego"]),
         PremiosUSD=lambda d: to_float(d["PremiosUSD"]),
         Torneos=lambda d: to_float(d["Torneos"])
     ))

# --- Esports general (dimensión del juego) ---
g_usecols = ["Game","Genre","TotalEarnings","TotalPlayers","TotalTournaments"]
g_dtypes  = {"Game":"string","Genre":"string",
             "TotalEarnings":"float64","TotalPlayers":"float64","TotalTournaments":"float64"}
g_raw = read_csv_fast(E_GEN, usecols=g_usecols, dtypes=g_dtypes)

g = (g_raw.rename(columns={
        "Game":"Juego","Genre":"Género",
        "TotalEarnings":"PremiosTotalesUSD",
        "TotalPlayers":"JugadoresTotales",
        "TotalTournaments":"TorneosTotales"
     })[["Juego","Género","PremiosTotalesUSD","JugadoresTotales","TorneosTotales"]]
     .assign(
         JuegoNorm=lambda d: apply_alias(d["Juego"]),
         Género=lambda d: norm_text(d["Género"]).str.title(),
         PremiosTotalesUSD=lambda d: to_float(d["PremiosTotalesUSD"]),
         JugadoresTotales=lambda d: to_float(d["JugadoresTotales"]),
         TorneosTotales=lambda d: to_float(d["TorneosTotales"])
     ))

# Downcast -> menor memoria
t, e, g = downcast_num(t), downcast_num(e), downcast_num(g)

print("✅ Datos cargados y traducidos")


⏱ read_csv_fast: 0.07s
🗓 Twitch años: 2016 → 2025
⏱ read_csv_fast: 0.01s
⏱ read_csv_fast: 0.02s
✅ Datos cargados y traducidos


In [103]:
# Tipado category para acelerar groupby
for dfc in (t, e, g):
    if "JuegoNorm" in dfc: dfc["JuegoNorm"] = dfc["JuegoNorm"].astype("category")
if "Género" in g: g["Género"] = g["Género"].astype("category")

# Twitch -> Juego–Año
t_ag = (t.groupby(["JuegoNorm","Año"], observed=True)
          .agg(MediaViewers=("MediaViewers","mean"),
               PicoViewers=("PicoViewers","max"),
               HorasVistas=("HorasVistas","sum"))
          .reset_index())

# Esports -> Juego–Año
e_ag = (e.groupby(["JuegoNorm","Año"], observed=True)
          .agg(PremiosUSD=("PremiosUSD","sum"),
               Torneos=("Torneos","sum"))
          .reset_index())

# Dimensión única por juego
g1 = (g.groupby("JuegoNorm", observed=True, as_index=False)
        .agg(Género=("Género", first_non_null),
             PremiosTotalesUSD=("PremiosTotalesUSD","max"),
             JugadoresTotales=("JugadoresTotales","max"),
             TorneosTotales=("TorneosTotales","max")))

# QA unicidad
assert not t_ag.duplicated(["JuegoNorm","Año"]).any()
assert not e_ag.duplicated(["JuegoNorm","Año"]).any()
assert not g1.duplicated("JuegoNorm").any()
print("✅ Agregaciones listas y únicas")



✅ Agregaciones listas y únicas


In [105]:
# Merge Twitch + Esports (1–a–1)
df = t_ag.merge(e_ag, on=["JuegoNorm","Año"], how="left", validate="one_to_one")

# Merge de dimensión (many–to–one)
df = df.merge(g1, on="JuegoNorm", how="left", validate="many_to_one")

# Anti-joins para depurar alias/nombres
no_esports_df = (t_ag.merge(e_ag, on=["JuegoNorm","Año"], how="left", indicator=True)
                   .query('_merge=="left_only"')[["JuegoNorm","Año"]]
                   .drop_duplicates()
                   .sort_values(["JuegoNorm","Año"]))
no_dim_df = (t_ag.merge(g1[["JuegoNorm"]], on="JuegoNorm", how="left", indicator=True)
               .query('_merge=="left_only"')[["JuegoNorm"]]
               .drop_duplicates()
               .sort_values("JuegoNorm"))

no_esports_df.to_csv(REVIEW/"anti_join_twitch_vs_esports.csv", index=False)
no_dim_df.to_csv(REVIEW/"anti_join_twitch_vs_dim.csv", index=False)

print("🔎 Sin match en Esports (muestra):", list(no_esports_df["JuegoNorm"].unique()[:15]))
print("🔎 Sin match en Dimensión (muestra):", list(no_dim_df["JuegoNorm"].unique()[:15]))
print("📝 Anti-joins guardados en /data/processed/review")
print("✅ Merges realizados")



🔎 Sin match en Esports (muestra): ['.hack//g.u. last recode', '20 minutes till dawn', '2xko', '60 parsecs!', '60 seconds!', '60 seconds! reatomized', '7 days to die', '<na>', '<u+014c>kami', 'a dance of fire and ice', 'a difficult game about climbing', 'a hat in time', 'a plague tale: innocence', 'a plague tale: requiem', 'a total war saga: troy']
🔎 Sin match en Dimensión (muestra): ['.hack//g.u. last recode', '20 minutes till dawn', '2xko', '60 parsecs!', '60 seconds!', '60 seconds! reatomized', '7 days to die', '<na>', '<u+014c>kami', 'a dance of fire and ice', 'a difficult game about climbing', 'a hat in time', 'a plague tale: innocence', 'a plague tale: requiem', 'a total war saga: troy']
📝 Anti-joins guardados en /data/processed/review
✅ Merges realizados


In [106]:
# Mantener SOLO juegos presentes en la dimensión g1 (universo eSports)
ESPORTS_UNIVERSE = set(g1["JuegoNorm"].unique())

to_drop = (df.loc[~df["JuegoNorm"].isin(ESPORTS_UNIVERSE), ["JuegoNorm","Año"]]
             .drop_duplicates()
             .sort_values(["JuegoNorm","Año"]))
to_drop_games = to_drop["JuegoNorm"].drop_duplicates().sort_values()

to_drop.to_csv(REVIEW/"purga_no_esports_rows.csv", index=False)
to_drop_games.to_csv(REVIEW/"purga_no_esports_titles.csv", index=False, header=["JuegoNorm"])

df = df[df["JuegoNorm"].isin(ESPORTS_UNIVERSE)].reset_index(drop=True)
print(f"✅ eSports-only; eliminados títulos: {to_drop_games.nunique()}")

# (Opcional) Exigir actividad competitiva anual (premios o torneos > 0):
mask = (df["PremiosUSD"].fillna(0) > 0) | (df["Torneos"].fillna(0) > 0)
df = df[mask].reset_index(drop=True); print("✅ + actividad anual aplicada")


✅ eSports-only; eliminados títulos: 2104
✅ + actividad anual aplicada


In [107]:
# Ventana temporal final
df = df[df["Año"].between(YEAR_MIN, YEAR_MAX)].copy()
df["Género"] = df["Género"].astype("string").fillna("Desconocido")

# Escalados legibles (conservando brutos para otros análisis)
if "MediaViewers" in df:       df["MediaViewers_K"]      = (df["MediaViewers"]/1_000).round(1)
if "PicoViewers" in df:        df["PicoViewers_K"]       = (df["PicoViewers"]/1_000).round(1)
if "HorasVistas" in df:        df["HorasVistas_M"]       = (df["HorasVistas"]/1_000_000).round(1)
if "PremiosUSD" in df:         df["PremiosUSD_M"]        = (df["PremiosUSD"]/1_000_000).round(2)
if "PremiosTotalesUSD" in df:  df["PremiosTotalesUSD_M"] = (df["PremiosTotalesUSD"]/1_000_000).round(2)

# Orden + limpieza (solo columnas esperadas)
df = (df[[c for c in COLS_YEARLY if c in df.columns]]
        .drop_duplicates(subset=["JuegoNorm","Año"])
        .sort_values(["Año","JuegoNorm"])
        .reset_index(drop=True))

# Tipado final optimizado
df["JuegoNorm"] = df["JuegoNorm"].astype("category")
df["Género"]    = df["Género"].astype("category")

print("✅ Dataset curado y escalado")
display(df.head())


✅ Dataset curado y escalado


,JuegoNorm,Año,Género,MediaViewers_K,PicoViewers_K,HorasVistas_M,PremiosUSD_M,Torneos,PremiosTotalesUSD_M,JugadoresTotales,TorneosTotales
0,age of empires,2016,Strategy,0.3,107.500000,0.2,0.06,22.0,0.74,624.0,341.0
1,age of empires ii,2016,Strategy,0.2,9.100000,1.5,0.06,41.0,3.90,2256.0,1939.0
2,battlefield 4,2016,First-Person Shooter,0.9,482.899994,6.4,0.01,3.0,0.25,204.0,145.0
3,battlerite,2016,Multiplayer Online Battle Arena,1.2,29.000000,3.6,0.01,38.0,0.07,88.0,76.0
4,blade & soul,2016,Role-Playing Game,1.5,67.300003,13.4,0.16,2.0,0.68,89.0,7.0


In [108]:
print("Duplicados por clave:", df.duplicated(["JuegoNorm","Año"]).sum())
print("Cobertura por año:\n", df.groupby("Año").size())
print("Juegos únicos:", df["JuegoNorm"].nunique())

# NaNs clave
for c in ["JuegoNorm","Año","MediaViewers_K"]:
    na = df[c].isna().mean()
    print(f"% NA {c}: {na:.2%}")

# Describe numérico robusto
try:
    desc = df.describe(numeric_only=True)
except TypeError:
    desc = df.select_dtypes(include=[np.number]).describe()

focus = [c for c in ["MediaViewers_K","PicoViewers_K","HorasVistas_M","PremiosUSD_M"] if c in desc.columns]
display(desc[focus].T)  # transpuesto: legible


Duplicados por clave: 0
Cobertura por año:
 Año
2016    53
2017    56
2018    62
2019    69
2020    68
2021    60
2022    60
2023    51
2024    39
dtype: int64
Juegos únicos: 191
% NA JuegoNorm: 0.00%
% NA Año: 0.00%
% NA MediaViewers_K: 0.00%


,count,mean,std,min,25%,50%,75%,max
MediaViewers_K,518.0,14.711196,29.703957,0.1,1.00,2.30,9.775000,198.399994
PicoViewers_K,518.0,241.957718,439.574219,1.5,35.55,84.50,205.425003,3291.500000
HorasVistas_M,518.0,112.964672,248.917943,0.1,3.80,14.30,62.275000,1735.700000
PremiosUSD_M,518.0,2.448784,6.714428,0.0,0.05,0.32,1.647500,80.690000


In [109]:
df_kpis = (df.groupby("JuegoNorm", observed=True, as_index=False)
             .agg(Género=("Género", first_non_null),
                  MediaViewers_K=("MediaViewers_K","mean"),
                  PicoViewers_K=("PicoViewers_K","max"),
                  HorasVistas_M=("HorasVistas_M","sum"),
                  PremiosUSD_M=("PremiosUSD_M","sum"),
                  Torneos=("Torneos","sum"),
                  PremiosTotalesUSD_M=("PremiosTotalesUSD_M","max"),
                  AñosActivos=("Año","nunique"))
           .sort_values(["PremiosUSD_M","MediaViewers_K","PicoViewers_K"], ascending=[False,False,False])
           .reset_index(drop=True))

display(df_kpis.head(12))


,JuegoNorm,Género,MediaViewers_K,PicoViewers_K,HorasVistas_M,PremiosUSD_M,Torneos,PremiosTotalesUSD_M,AñosActivos
0,dota 2,Multiplayer Online Battle Arena,58.922222,1636.400024,4503.8,304.65,1323.0,360.46,9
1,fortnite,Battle Royale,102.757141,3291.500000,6143.0,191.01,2293.0,191.01,7
2,counter-strike: global offensive,First-Person Shooter,68.375000,1916.000000,4315.6,152.84,5794.0,162.49,8
3,league of legends,Multiplayer Online Battle Arena,147.522217,3145.000000,11296.8,86.68,1297.0,113.60,9
4,rocket league,Sports,10.655556,408.500000,816.5,42.40,4477.0,42.26,9
5,overwatch,First-Person Shooter,23.485714,1459.199951,1393.5,35.26,837.0,35.26,7
6,valorant,First-Person Shooter,117.619995,1729.000000,4631.9,32.03,1316.0,32.02,5
7,hearthstone,Collectible Card Game,32.333332,331.200012,2513.2,27.20,730.0,30.71,9
8,apex legends,Battle Royale,52.916668,666.900024,2655.2,26.14,731.0,26.15,6
9,starcraft ii,Strategy,5.066667,148.899994,394.0,24.06,3918.0,42.69,9


In [110]:
PROC.mkdir(parents=True, exist_ok=True)

# CSV
df.to_csv(PROC/"Esports_Occidente_Master.csv", index=False)
df_kpis.to_csv(PROC/"Esports_Occidente_KPIs.csv", index=False)

# Parquet (rápido para recargas)
try:
    df.to_parquet(PROC/"Esports_Occidente_Master.parquet", index=False)
    df_kpis.to_parquet(PROC/"Esports_Occidente_KPIs.parquet", index=False)
    print("🗂 CSV + Parquet guardados")
except Exception as e:
    print("⚠️ Parquet no disponible:", e)

# Dimensión de juegos para curación/merge externo
g1.rename(columns={"JuegoNorm":"game_norm","Género":"genero"}).to_csv(PROC/"games_dim.csv", index=False)
print("✅ Artefactos exportados a data/processed")


🗂 CSV + Parquet guardados
✅ Artefactos exportados a data/processed


In [111]:
def view(dataframe, year=None, game=None, genre=None, cols=None, sort=None):
    out = dataframe
    if year is not None:  out = out[out["Año"].eq(year)]
    if game is not None:  out = out[out["JuegoNorm"].astype("string").str.contains(game, case=False, na=False)]
    if genre is not None: out = out[out["Género"].astype("string").str.contains(genre, case=False, na=False)]
    if cols: out = out[cols]
    if sort: out = out.sort_values(sort)
    return out

# Ejemplos
display(view(df, year=df["Año"].max()).head(5))
display(view(df_kpis, game="league of legends"))
display(view(df_kpis, genre="Shooter").head(5))


,JuegoNorm,Año,Género,MediaViewers_K,PicoViewers_K,HorasVistas_M,PremiosUSD_M,Torneos,PremiosTotalesUSD_M,JugadoresTotales,TorneosTotales
479,age of empires ii,2024,Strategy,2.6,98.099998,16.9,0.50,233.0,3.90,2256.0,1939.0
480,age of empires iv,2024,Strategy,1.1,40.000000,4.8,0.15,58.0,1.19,643.0,423.0
481,age of mythology,2024,Strategy,3.1,27.200001,2.2,0.03,23.0,0.19,236.0,298.0
482,apex legends,2024,Battle Royale,36.5,341.700012,239.4,4.25,6.0,26.15,2317.0,731.0
483,brawl stars,2024,Strategy,4.0,135.500000,26.4,0.05,1.0,8.14,698.0,207.0


,JuegoNorm,Género,MediaViewers_K,PicoViewers_K,HorasVistas_M,PremiosUSD_M,Torneos,PremiosTotalesUSD_M,AñosActivos
3,league of legends,Multiplayer Online Battle Arena,147.522217,3145.000000,11296.8,86.68,1297.0,113.60,9
11,league of legends: wild rift,Multiplayer Online Battle Arena,2.150000,60.400002,73.0,19.48,203.0,11.19,4


,JuegoNorm,Género,MediaViewers_K,PicoViewers_K,HorasVistas_M,PremiosUSD_M,Torneos,PremiosTotalesUSD_M,AñosActivos
2,counter-strike: global offensive,First-Person Shooter,68.375000,1916.000000,4315.6,152.84,5794.0,162.49,8
5,overwatch,First-Person Shooter,23.485714,1459.199951,1393.5,35.26,837.0,35.26,7
6,valorant,First-Person Shooter,117.619995,1729.000000,4631.9,32.03,1316.0,32.02,5
13,call of duty: warzone,First-Person Shooter,63.359997,1629.300049,2681.5,12.91,406.0,12.90,5
14,overwatch 2,First-Person Shooter,33.100002,634.000000,646.7,12.83,135.0,12.86,3
